# Importation des bibliothèques

In [1]:
%%capture
## Instalation des bibliothèques utilitaires
!pip install google-api-python-client
!pip install --upgrade google-auth-httplib2 google-auth-oauthlib

In [2]:
%%capture

import warnings

warnings.filterwarnings("ignore")

from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
from openhexa.sdk import workspace

pd.set_option("display.max_columns", None)

try:
    import openpyxl as pyxl
    from fuzzywuzzy import fuzz, process
except ImportError or ModuleNotFoundError:
    !pip install python-Levenshtein
    !pip install fuzzywuzzy
    from fuzzywuzzy import fuzz, process
    import openpyxl as pyxl

import os
from functools import partial
from importlib import reload

In [3]:
# Ajout du chemin d'accès aux différents code
os.chdir(Path(workspace.files_path, "Rapport Feedback/code/pipelines"))

# Module créer pour le processing et l'exportation des données
from compute_indicators import compute_indicators, date_utils, excel_file_handler
from database_operations import db_ops, update_dimension, upsert_table
from export_file_to_google_drive import upload_file_to_drive
from generate_feedback_report import generate_feedback_report as gfr
from metabase import queries
from metabase.metabase import Metabase

# Définition des paramètres

Les variables nécessaires à la production du rapport de feedback :

1. **Mois de création du rapport**  
   - **Description :** Ce paramètre correspond au mois pour lequel le rapport de feedback sera généré.  
   - **Source :** Défini par l'utilisateur lors de l'exécution du pipeline.

2. **Liste des sites attendus**  
   - **Description :** Fichier Excel contenant la liste complète des sites devant être inclus dans le rapport.  
   - **Fréquence de mise à jour :** Ce fichier est mis à jour et communiqué chaque trimestre.  
   - **Emplacement requis :** Le fichier doit être déposé dans le dossier `Rapport Feedback/data/Sites attendus`.

3. **Liste des produits traceurs**  
   - **Description :** Fichier Excel fournissant les informations sur les produits traceurs à analyser dans le rapport.  
   - **Fréquence de mise à jour :** Généralement communiqué mensuellement par la DAP.  
   - **Emplacement requis :** Le fichier doit être placé dans le dossier `Rapport Feedback/data/Produits Traceurs`.

<div class="alert alert-block alert-success">
Ici quand le mois est défini il faudrait aller checker dans la base de données pour voir si le mois en question est déjà présent dans la base pour minimiser le risque d'erreur
</div>

In [4]:
date_report, fp_site_attendus, fp_prod_traceurs = (
    "Mars",
    "Sites attendus Février 2025.xlsx",
    "Liste des Produits Traceurs Février 2025.xlsx",
)

In [5]:
# Parameters
date_report = "Avril"
fp_site_attendus = "Sites attendus 2025 Version Avril 2025.xlsx"
fp_prod_traceurs = "Liste des Produits Traceurs F\xe9vrier 2025.xlsx"


In [6]:
month_export, date_report = date_report, compute_indicators.generate_month_end_report_date(date_report)

## Test pour s'assurer que cette date n'existe pas déjà dans la base de données

In [7]:
db_ops.reload_connection()

schema_name = "dap_tools"

In [8]:
# Ici il faudrait également s'assurer que le mois précédent est bien présent à l'intérieur de la base de données au cas où on arriverait à se tromper sur le mois de conception du fichier
mois_prec = (pd.to_datetime(date_report).replace(day=1) - pd.Timedelta(days=1)).strftime('%Y-%m-%d')

df_ = pd.read_sql(f"select * from {schema_name}.etat_de_stock where date_report='{mois_prec}'", db_ops.civ_engine)

assert df_.shape[0] != 0, f"Le mois précédent {mois_prec} n'est pas présent dans la base de données locale êtes-vous sûre d'avoir choisir le bon mois de conception du fichier."

del df_

In [9]:
%%script false --no-raise-error

df_ = pd.read_sql(f"select * from {schema_name}.etat_de_stock where date_report='{pd.to_datetime(date_report).strftime('%Y-%m-%d')}' LIMIT 2", db_ops.civ_engine)

assert df_.shape[0] == 0, f"La date {date_report} existe déjà dans la base de données vous devez définir une autre date pour la production du rapport."

del df_

# 📥 Importation des Données
  
L'utilisateur doit veiller à ce que les fichiers respectent le format attendu et soient placés dans les répertoires dédiés avant de procéder au traitement.

## 📌 Importation de la `Liste des sites attendus`
- **Format requis :** Assurez-vous que le fichier respecte le template standard défini pour les mois de chargement.
- **Emplacement du fichier :** Le fichier doit être placé dans le répertoire dédié :  
  **`Rapport Feedback/data/Sites attendus`**  
- **En cas d'erreur :**  
  - Vérifiez que le fichier est bien présent dans le répertoire.  
  - Assurez-vous que toutes les colonnes requises sont bien renseignées.  
  - Contrôlez que le fichier est bien accessible et non corrompu.

In [10]:
fp_site_attendus = (
    Path(workspace.files_path)
    / "Rapport Feedback/data/Sites attendus" 
    / Path(fp_site_attendus).name
)

In [11]:
# Configuration constante
EXPECTED_COLS = {
    'Code', 'Site', 'District', 'Region', 'ARV', 'TRC', 'LAB', 
    'CHARGE VIRALE', 'PNLP', 'PNSME', 'PNSME-GRAT', 
    'PNN', 'TBS', 'TBMR', 'TBLAB'
}

# Utilisation
try:
    df_site_attendu = excel_file_handler.load_expected_sites_from_excel(fp_site_attendus, EXPECTED_COLS)
except Exception as e:
    print("Une erreur s'est produite lors du chargement du fichier contenant la liste des sites attendus.")
    print("Veuillez vérifier que le fichier existe dans le répertoire spécifique `Rapport Feedback/data/Sites attendus` et qu'il contient toutes les colonnes requises.")
    print(f"Code d'erreur détaillé : {e}")
    raise
df_site_attendu.head(2)

,Code,Site,District,Region,ARV,TRC,LAB,CHARGE VIRALE,PNLP,PNN,TBS,TBMR,TBLAB,PNSME-GRAT,PNSME
0,10200010,CHU YOPOUGON,YOPOUGON OUEST-SONGON,ABIDJAN 1,1,1,NA,NA,NA,1,NA,NA,NA,NA,NA
1,10300010,CHU ANGRE,COCODY-BINGERVILLE,ABIDJAN 2,1,1,NA,1,1,1,1,NA,NA,1,NA


## 📌 Importation de la `liste des produits traceurs`
- **Format requis :** Respectez la structure du fichier définie pour l’importation.  
- **Emplacement du fichier :** Le fichier doit être placé dans le répertoire suivant :  
  **`Rapport Feedback/data/Produits Traceurs`**  
- **En cas d'erreur :**  
  - Vérifiez que le fichier est bien présent dans le répertoire.  
  - Assurez-vous que les données respectent le format attendu.  
  - Vérifiez l'intégrité du fichier et assurez-vous et corriger l'anomalie.

In [12]:
# Il faudra définir un standard de fichier qui sera attendu
fp_prod_traceurs = (
    Path(workspace.files_path)
    / "Rapport Feedback/data/Produits Traceurs" 
    / Path(fp_prod_traceurs).name
)

In [13]:
try:
    df_prod_traceurs = excel_file_handler.load_traceable_products_from_excel(fp_prod_traceurs)
except Exeption as e:
    print("Une erreur s'est produite lors du chargement du fichier contenant la liste des produits traceurs")
    print("Veuillez vérifier que le fichier existe et qu'il contienu dans le répertoire dédié `Rapport Feedback/data/Produits Traceurs`")
    print(f"Code d'erreur détaillé : {e}")

df_prod_traceurs.head(2)

,PROGRAMME,CODE PRODUIT,PRODUIT,CODE COMBINE,CATEGORIE PRODUIT
0,PNLP,3050015,"AMODIAQUINE/ARTESUNATE 25 / 67,5 mg ENFANT (0 ...",3050015_PNLP,Produit traceur
1,PNLP,3050016,AMODIAQUINE/ARTESUNATE 50 / 135 mg ENFANT (1 -...,3050016_PNLP,Produit traceur


## 📌 Extraction des données eSIGL : `État de transmission`  

Cette étape permet d'extraire les données d'état de transmission depuis eSIGL via Metabase.  
Assurez-vous que toutes les conditions nécessaires sont remplies avant de lancer l'extraction.  

🛠 En cas d'erreur :  
1. **Vérifier les permissions utilisateur**  
   - Assurez-vous que l'utilisateur **`secretariat_dap`** dispose bien des accès nécessaires à eSIGL.  
   - Si l'accès est restreint, contactez l'administrateur du système pour obtenir les autorisations requises.  

2. **S'assurer du bon fonctionnement de eSIGL**  
   - Il arrive que **Metabase** subisse des interruptions temporaires rendant l'accès aux données impossible.  
   - Si eSIGL ou Metabase est indisponible, réessayez plus tard ou consultez l'équipe technique.  

**Astuce :** Si le problème persiste, tentez d'accéder manuellement à eSIGL et Metabase pour vérifier leur disponibilité.

In [14]:
metabase = Metabase(workspace.custom_connection("metabase-esigl"))

In [15]:
df_transmission = metabase.get_data_from_sql_query(
    queries.QUERY_TRANSMISSION.format(
        date_report=date_utils.get_date_report(date_report)
    )
)

# Les établissements ne sont plus censé faire des rapportages sur ce programme spécifique
df_transmission["program"] = df_transmission["program"].str.replace(
    "PNSME-MEDICAMENTS ET INTRANTS", "PNSME_GRATUITE:MEDICAMENTS ET INTRANTS"
)

df_transmission.head(3)

,region,id_region_esigl,code,facility,district,id_district_esigl,program,cde_urgente,period,statut,user,date_soumission,date_autorisation,time_ago
0,BOUNKANI,6,50800050,DISTRICT SANITAIRE NASSIAN,NASSIAN,77,PNLS-ANTIRETROVIRAUX ET IO,False,AVRIL 2025,AUTHORIZED,BROUHA KOUASSI MATHIEU,2025-05-10T00:00:00Z,2025-05-10T00:00:00Z,1 09:44:05
1,ABIDJAN 1,2,10200010,CHU YOPOUGON,YOPOUGON EST,101,PNLS-ANTIRETROVIRAUX ET IO,False,AVRIL 2025,AUTHORIZED,ADEHOUNI YACOUBA A.,2025-05-08T00:00:00Z,2025-05-09T00:00:00Z,2 01:55:19
2,GONTOUGO,144,50800040,DISTRICT SANITAIRE KOUN FAO,KOUN FAO,119,PNN-MEDICAMENTS ET INTRANTS,False,AVRIL 2025,APPROVED,Kouame Mbia Moussa,2025-05-10T00:00:00Z,2025-05-10T00:00:00Z,1 08:12:36


## 📌 Extraction des données eSIGL : `Etat de stock`

In [16]:
df_etat_stock = metabase.get_data_from_sql_query(
    queries.QUERY_ETAT_STOCK.format(date_report=date_utils.get_date_report(date_report))
)

df_etat_stock["programme"] = df_etat_stock["programme"].str.replace(
    "PNSME-MEDICAMENTS ET INTRANTS", "PNSME_GRATUITE:MEDICAMENTS ET INTRANTS"
)

df_etat_stock.head(3)

,commande_urgente,programme,periode,region,id_region_esigl,district,id_district_esigl,code,etablissement,type_structure,categorie_produit,code_produit,designation,unite,stock_initial,quantite_recue,quantite_distribuee,perte_ajustement,sdu,cmm,nbrejrsrupture,quantite_proposee,quantite_commandee,quantite_approuvee,explication_de_la_qte_cmdee
0,False,PNSME_GRATUITE:MEDICAMENTS ET INTRANTS,AVRIL 2025,ABIDJAN 1,2,YOPOUGON EST,101,60100090,CENTRE DE SANTE MIXTE DE YOPOUGON MILLIONNAIRE...,PUBLIC,INJECTABLES,3120015,JADELLE (IMPLANT + TROCARD) 150 mg BTE/10 BOI...,IMPLANT,10,0,3,-2,5,2,0,3,7,7,CMM GESTIONNAIRE=3
1,False,PNSME_GRATUITE:MEDICAMENTS ET INTRANTS,AVRIL 2025,ABIDJAN 1,2,YOPOUGON EST,101,60100090,CENTRE DE SANTE MIXTE DE YOPOUGON MILLIONNAIRE...,PUBLIC,INJECTABLES,3120048,"SAYANA PRESS (DMPA-SC) 104mg/0,65ml amp inj UN -",AMPOULE,22,0,3,0,19,2,0,0,0,0,CMM=2
2,False,PNSME_GRATUITE:MEDICAMENTS ET INTRANTS,AVRIL 2025,ABIDJAN 1,2,YOPOUGON EST,101,60100090,CENTRE DE SANTE MIXTE DE YOPOUGON MILLIONNAIRE...,PUBLIC,INJECTABLES,3120034,NORISTERAT 200 mg amp. inj. BTE/100 BTE -,AMPOULE,0,0,0,0,0,0,30,0,100,100,MISE EN PLACE


# 📊 Calcul des Indicateurs `(1/3)`

Cette étape consiste à **traiter les données du mois en cours** afin de calculer les indicateurs de performance, notamment :  
- **Complétude** : mesure le taux de soumission des données attendues.  
- **Promptitude** : évalue la rapidité avec laquelle les données sont transmises.
- **Etat de Stock**: informations sur les états de stock des différents produits.

Une fois calculés, ces indicateurs sont intégrés au **rapport de feedback**, qui sera ensuite exporté pour être accessible aux utilisateurs.

In [17]:
%%time
df_ets, df_region = compute_indicators.compute_indicators_completeness_and_promptness(
    df_site_attendu.copy(), df_transmission.copy(), date_report
)

CPU times: user 7.11 s, sys: 6.49 ms, total: 7.12 s
Wall time: 7.54 s


## 📄 Génération du Fichier Excel `(1/2)`

Cette étape permet de **générer un fichier Excel** contenant les indicateurs calculés et les données consolidées du rapport de feedback.  
Une fois le fichier créé, un **lien d’accès** est intégré dans le tableau de bord afin que les utilisateurs puissent le consulter et le télécharger facilement.  

In [18]:
src_file = (
    Path(workspace.files_path)
    / "Rapport Feedback/data/Template Rapport Feedback/RAPPORT FEEDBACK - TEMPLATE.xlsx"
)

wb_feedback_report = gfr.load_workbook(src_file.as_posix())

In [19]:
%%time
gfr.export_detail_comp_promp_to_sheet(
    wb_feedback_report, df_ets.copy(), df_region.copy(), date_report
)

CPU times: user 2.49 s, sys: 2.96 ms, total: 2.5 s
Wall time: 2.5 s


## 📊 Suite du Calcul des Indicateurs `(2/3)`

Cette étape poursuit le traitement des données en complétant le calcul des indicateurs clés pour le rapport de feedback.  
Elle permet de finaliser les analyses et d’assurer la cohérence des résultats avant leur exportation.  

In [20]:
%%time
(
    df_etat_stock,
    stock_lvl_decent,
    stock_region,
) = compute_indicators.analyze_product_stock_status_indicators(
    df_prod_traceurs.copy(), df_etat_stock.copy(), date_report
)

CPU times: user 43.7 s, sys: 23.3 ms, total: 43.7 s
Wall time: 43.9 s


## 📄 Génération Finale du Fichier Excel `(2/2)`


In [21]:
%%time
gfr.export_stock_data_to_sheet(wb_feedback_report, df_etat_stock.copy())

CPU times: user 1min 56s, sys: 898 ms, total: 1min 57s
Wall time: 1min 57s


In [22]:
%%time
# Date report doit être révue pour prendre le 11 du mois en cours
gfr.export_stock_region_to_sheet(
    wb_feedback_report,
    stock_lvl_decent.copy(),
    stock_region.copy(),
    date_report=pd.to_datetime(date_report).replace(day=11).strftime("%Y/%m/%d"),
)

CPU times: user 8.88 s, sys: 12.9 ms, total: 8.9 s
Wall time: 8.9 s


In [23]:
# Sauvegarde du fichier dans un repertoire courant
dest_file = (
    Path(workspace.files_path)
    / f"Rapport Feedback/code/pipelines/rapport feedback genere/{date_report[:4]}"
)
dest_file.mkdir(exist_ok=True, parents=True)

dest_file = dest_file / f"RapportFeedBack-{month_export.upper()}-{date_report[:4]}.xlsx"

wb_feedback_report.save(dest_file.as_posix())

del wb_feedback_report

## 📊 Suite du Calcul des Indicateurs `(3/3)`

In [24]:
%%time
(
    stock_lvl_decent,
    stock_region,
    df_sheet_two,
    stock_region_with_central,
) = compute_indicators.aggregate_regional_stock_availability_metrics(
    stock_lvl_decent.copy(), stock_region.copy()
)

CPU times: user 1.38 s, sys: 1.98 ms, total: 1.38 s
Wall time: 1.4 s


# Exportation des données vers la base des données

## Export du lien du fichier vers le répertoire drive

In [25]:
share_link = upload_file_to_drive.upload_file_and_get_share_link(
    dest_file.as_posix(), date_report=date_report
)

df_share_link = pd.DataFrame(
    data=[{"share_link": share_link, "date_report": pd.to_datetime(date_report)}]
)

db_ops.civ_cursor.execute(
    f"""
DELETE FROM {schema_name}.share_link_fbr
WHERE date_report = '{date_report}'
"""
)

db_ops.conn.commit()

df_share_link.to_sql(
    "share_link_fbr",
    con=db_ops.civ_engine,
    schema=schema_name,
    index=False,
    if_exists="append",
)

db_ops.civ_engine.dispose()

del df_share_link

Aucun file trouvé avec ce nom.


## Exportation des autres tables

In [26]:
# completude_promptitude_par_ets
df_comp_promp_ets = df_ets.rename(
    columns=lambda x: x.lower().replace(" ", "_").replace("-", "_")
    if x not in ("Code", "Site", "Region")
    else x
)

# completude_promptitude_attendu_taux_region
df_comp_promp_region = df_region.rename(
    columns=lambda x: x.lower().replace(" ", "_") if x not in ("Region") else x
)

In [27]:
date_report = pd.to_datetime(date_report).strftime("%d-%m-%Y")

In [28]:
df_etat_stock["date_report"] = pd.to_datetime(date_report, format="%d-%m-%Y")
stock_region_with_central["date_report"] = pd.to_datetime(
    date_report, format="%d-%m-%Y"
)
df_sheet_two["date_report"] = pd.to_datetime(date_report, format="%d-%m-%Y")

# Some change here
df_comp_promp_ets["date_report"] = pd.to_datetime(date_report, format="%d-%m-%Y")
df_comp_promp_region["date_report"] = pd.to_datetime(date_report, format="%d-%m-%Y")

In [29]:
df_etat_stock.rename(
    columns={
        "CODE": "Code_produit",
        "PROGRAMME": "programme_abrv",
        "SOUS-PROGRAMME": "sous_programme",
        "PERIODE": "Periode",
        "REGION": "Region",
        "DISTRICT": "District",
        "CODE ETS": "Code_ets",
        "STRUCTURE": "Structure",
        "CATEGORIE PRODUIT": "cat_produit",
        "PRODUIT": "produit_designation",
        "UNITE DE RAPPORTAGE": "unit_rapportage",
        "STOCK INITIAL": "stock_initial",
        "QUANTITE RECUE": "qte_recue",
        "QUANTITE UTILISEE": "qte_utilisee",
        "PERTES ET AJUSTEMENT": "perte_ajust",
        "JOURS DE RUPTURE": "j_rupture",
        "SDU": "sdu",
        "CMM ESIGL": "cmm_esigl",
        "CMM gestionnaire": "cmm_gest",
        "QUANTITE PROPOSEE": "qte_prop",
        "QUANTITE COMMANDEE": "qte_cmde",
        "QUANTITE APPROUVEE": "qte_approuv",
        "MSD": "msd",
        "ETAT DU STOCK": "etat_stock",
        "BESOIN CMMMANDE URGENTE": "besoin_cmde_urg",
        "BESOIN TRANSFERT IN": "besoin_trsf_in",
        "QUANTITE A TRANSFERER OUT": "qte_trsf_out",
        "CATEGORIE_DU_PRODUIT": "cat_du_produit",
    }, inplace=True)

stock_region_with_central.rename(columns={"Code": "Code_produit", "MSD": "msd", "STATUT": "Statut"}, inplace=True)

### Mise à jour des dimensions

#### `Dimensions régions`


Cette étape permet de maintenir à jour la **dimension des régions** dans la base de données.  
Elle garantit que toutes les informations régionales sont alignées avec les mises à jour effectuées dans **eSIGL**

 🛠 Actions réalisées :  
1. **Mise à jour des régions existantes**  
   - Si des modifications ont été apportées aux régions dans **eSIGL**, elles sont répercutées dans notre base de données.  
   - Les informations régionales (ex. : noms, codes, identifiants) sont mises à jour pour rester cohérentes avec la source officielle.  

2. **Ajout des nouvelles régions**  
   - Si certaines régions ne sont pas encore présentes dans notre base, elles sont automatiquement ajoutées.  
   - Cela garantit que l’ensemble des régions récentes est bien pris en compte dans les analyses et traitements futurs.  
  

In [30]:
# Recherche des nouvelles régions et modifications sur les régions survenues au cours de la période dans eSIGL
df_new_region = update_dimension.update_dimension_table(
    dimension_name="dim_region",
    source_dfs=[
        df_transmission[["region", "id_region_esigl"]].rename(
            columns={"region": "Region"}
        ),
        df_etat_stock[["Region", "id_region_esigl"]],
    ],
    merge_on=["id_region_esigl"],
    change_columns=["Region"],
    code_generation=partial(update_dimension.region_code_generation, metabase=metabase),
    schema_name=schema_name,
)

df_new_region.head(3)

,Region,id_region_esigl,Code_region,Region_existing,region_order


In [31]:
# Mise à jour des informations au niveau de la base de données
upsert_table.upsert_table(
    df_new_region, "dim_region", schema_name, engine=db_ops.civ_engine
)

del df_new_region

In [32]:
df_region = db_ops.get_data_from_database("dim_region")

df_region.head(2)

,Code_region,Region,region_order,id_region_esigl
0,NAT,NATIONAL,1,NaN
1,N-PSP,NOUVELLE PSP,0,NaN


#### `Dimension districts`

Mise à Jour et Ajout d'Informations Complémentaires  

Avant d'exécuter le traitement, il est **essentiel** de mettre à jour les informations si des modifications ont été effectuées dans **eSIGL**, notamment sur les districts.  

Étapes à suivre :  
1. **Vérification et mise à jour des informations existantes**  
   - Si des districts ont été modifiés ou ajoutés dans **eSIGL**, il faut s'assurer que ces changements sont bien répercutés dans les données utilisées.  

2. **Ajout des informations complémentaires**  
   - Une fois la mise à jour effectuée, il est nécessaire d'ajouter toutes les informations additionnelles requises pour le bon fonctionnement du processus.

In [33]:
df_new_district = update_dimension.update_dimension_table(
    dimension_name="dim_district",
    source_dfs=[
        df_transmission[["id_region_esigl", "district", "id_district_esigl"]].rename(
            columns={"district": "District"}
        ),
        df_etat_stock[["id_region_esigl", "District", "id_district_esigl"]],
    ],
    merge_on=["id_district_esigl"],
    change_columns=["District"],
    code_generation=partial(
        update_dimension.district_code_generation,
        tb_region="dim_region",
        schema_name=schema_name,
    ),
    schema_name=schema_name,
)

In [34]:
upsert_table.upsert_table(
    df_new_district, "dim_district", schema_name, engine=db_ops.civ_engine
)

del df_new_district

In [35]:
df_district_db = db_ops.get_data_from_database("dim_district")

#### `Dimensions Structure`

##### Première mise à jour de la dimensions structure avec les données eSIGL

In [36]:
df_new_structure = update_dimension.update_dimension_table(
    dimension_name="dim_structure",
    source_dfs=[
        df_etat_stock[
            ["Code_ets", "Structure", "TYPE DE STRUCTURE", "id_district_esigl"]
        ].rename(columns={"TYPE DE STRUCTURE": "type_structure"})
    ],
    merge_on=["Code_ets"],
    change_columns=["Structure", "type_structure"],
    schema_name=schema_name,
)

In [37]:
# Exportation des données vers la BD
upsert_table.upsert_table(
    df_new_structure, "dim_structure", schema_name, engine=db_ops.civ_engine
)

del df_new_structure

##### Seconde mise à jour avec la liste des sites attendus

In [38]:
df_site_attendu["District_standard"] = df_site_attendu["District"].apply(
    lambda x: excel_file_handler.standardize_text(x)
)

df_district_db = db_ops.get_data_from_database("dim_district")
df_structure_db = db_ops.get_data_from_database("dim_structure")
df_structure_db["Code_ets"] = df_structure_db["Code_ets"].astype("Int64")

df_district_db["District"] = df_district_db["District"].apply(
    lambda x: excel_file_handler.standardize_text(x)
)

df_site_attendu = df_site_attendu.merge(
    df_district_db,
    left_on="District_standard",
    right_on="District",
    suffixes=("", "_existing"),
)

df_new_structure = (
    df_site_attendu[["Code", "Site", "id_district_esigl"]]
    .drop_duplicates()
    .rename(columns={"Code": "Code_ets", "Site": "Structure"})
)

df_new_structure = df_new_structure.loc[
    ~df_new_structure["Code_ets"].isin(df_structure_db["Code_ets"])
]

df_new_structure = df_new_structure.merge(df_district_db, on="id_district_esigl")
df_new_structure["type_structure"] = np.nan

df_new_structure = df_new_structure[df_structure_db.columns.to_list()]

In [39]:
# Exportation des données vers la BD
upsert_table.upsert_table(
    df_new_structure, "dim_structure", schema_name, engine=db_ops.civ_engine
)

del df_new_structure

In [40]:
df_structure_db = db_ops.get_data_from_database("dim_structure")

#### `Dimensions programme`

In [41]:
prog_extract_stock = set(df_etat_stock.programme_abrv.unique()).union(["TOUS"])

df_programme = (
    pd.DataFrame({"Programme": list(prog_extract_stock)})
    .sort_values(by="Programme")
    .reset_index()
    .drop(columns="index")
)
df_programme.dropna(inplace=True)

del prog_extract_stock

# Pour l'instant ce n'est que les produits des 5 programmes de santé qui sont gérés
programme_order = {"PNLS": 1, "PNLP": 2, "PNSME": 3, "PNN": 4, "PNLT": 5, "TOUS": 6}
df_programme["programme_order"] = df_programme["Programme"].map(programme_order)
df_programme.sort_values("programme_order", inplace=True)
del programme_order

df_programme = db_ops.get_full_table(
    df_programme,
    "dim_programme",
)

df_programme

,Programme,programme_order
0,PNLS,1
1,PNLP,2
2,PNSME,3
3,PNN,4
4,PNLT,5
5,TOUS,6


#### `Dimension sous_programme`

In [42]:
df_sous_prog_db = db_ops.get_data_from_database(
    "dim_sous_programme",
)

df_sous_prog_db = df_sous_prog_db[["Programme", "Sous_programme"]].rename(
    columns={"Programme": "programme_abrv", "Sous_programme": "sous_programme"}
)

In [43]:
df_sous_prog = pd.concat(
    [
        df_etat_stock[["programme_abrv", "sous_programme"]].drop_duplicates(),
        df_sous_prog_db,
    ],
    ignore_index=True,
).drop_duplicates()

df_sous_prog = (
    df_sous_prog.sort_values(["programme_abrv", "sous_programme"])
    .reset_index()
    .drop(columns="index")
)

# df_sous_prog['Code_sous_prog'] =
df_sous_prog["Occurence"] = df_sous_prog.groupby("programme_abrv").cumcount() + 1
df_sous_prog["Code_sous_prog"] = (
    df_sous_prog["programme_abrv"] + "-" + df_sous_prog["Occurence"].astype(str)
)

df_sous_prog = df_sous_prog[
    ["Code_sous_prog", "sous_programme", "programme_abrv"]
].rename(columns={"sous_programme": "Sous_programme", "programme_abrv": "Programme"})
df_sous_prog = df_sous_prog.map(
    lambda x: x.lstrip().rstrip() if isinstance(x, str) else x
)

del df_sous_prog_db

In [44]:
df_sous_prog = db_ops.get_full_table(df_sous_prog, "dim_sous_programme", )

df_sous_prog.head(2)

,Code_sous_prog,Sous_programme,Programme
0,PNLP-1,PNLP-MEDICAMENTS ET INTRANTS,PNLP
1,PNLS-1,PNLS-ANTIRETROVIRAUX ET IO,PNLS


#### `Dimension produit`

In [45]:
df_new_product = (
    df_etat_stock[
        [
            "Code_produit",
            "sous_programme",
            "produit_designation",
            "unit_rapportage",
            "cat_du_produit",
            "cat_produit",
        ]
    ]
    .drop_duplicates()
    .rename(columns={"sous_programme": "Sous_programme"})
)

df_new_product = df_new_product.map(
    lambda x: x.lstrip().rstrip() if isinstance(x, str) else x
)

df_new_product = (
    df_new_product.merge(df_sous_prog, how="left", on="Sous_programme")
    .drop(columns=["Sous_programme", "Programme"])
    .rename(
        columns={
            "cat_du_produit": "Categorie_du_produit",
            "cat_produit": "Categorie_produit",
        }
    )
    .rename(columns=lambda x: x.capitalize())
    .sort_values("Code_produit")
    .reset_index(drop=True)
)

In [46]:
df_new_product["Code_produit"] = df_new_product["Code_produit"].astype(str)

df_product_db = db_ops.get_data_from_database("dim_produit")

df_new_product = df_new_product.merge(
    df_product_db,
    on=["Code_produit", "Code_sous_prog"],
    how="left",
    suffixes=("_new", "_past"),
)
condition = ""
for col in [
    col.replace("_past", "") for col in df_new_product.columns if "_past" in col
]:
    condition += f"(df_new_product.{col}_new != df_new_product.{col}_past) | "

condition = condition.rstrip("| ")

df_new_product = df_new_product.loc[eval(condition)]

if not df_new_product.empty:
    df_new_product = df_new_product.drop_duplicates(
        subset=[
            "Code_produit",
            "Unit_rapportage_new",
            "Categorie_du_produit_new",
            "Categorie_produit_new",
            "Code_sous_prog",
            "id_produit_pk",
        ],
        keep="last",
    )

    max_val = df_product_db["id_produit_pk"].max() + 1
    missing = df_new_product["id_produit_pk"].isna()
    df_new_product.loc[missing, "id_produit_pk"] = range(
        max_val, max_val + missing.sum()
    )

    df_new_product.columns = df_new_product.columns.str.replace("_new", "")

df_new_product = df_new_product[df_product_db.columns.to_list()]

df_new_product.head(2)

,id_produit_pk,Code_produit,Produit_designation,Unit_rapportage,Categorie_produit,Categorie_du_produit,Code_sous_prog
1667,5314.0,4030616,TEST DE DIAGNOSTIC RAPIDE DU PALUDISME KIT/10 ...,KIT,Produit traceur,CONSOMMABLES,PNLP-1
1668,5315.0,4030616,TEST DE DIAGNOSTIC RAPIDE DU PALUDISME KIT/10 ...,TEST,Produit traceur,CONSOMMABLES,PNLP-1


In [47]:
upsert_table.upsert_table(
    df_new_product, "dim_produit", schema_name, engine=db_ops.civ_engine
)

del df_new_product

### Table de faits `Complétude` et `Promptitude`

In [48]:
df_region = db_ops.get_data_from_database("dim_region")

#### `df_comp_promp_ets`

In [49]:
db_ops.civ_cursor.execute(
    f"""
DELETE FROM {schema_name}.comp_promp_par_ets
WHERE date_report = '{pd.to_datetime(date_report, format="%d-%m-%Y").strftime('%Y-%m-%d')}'
"""
)

db_ops.conn.commit()

df_comp_promp_ets.drop(columns=["Site", "Region"]).rename(
    columns={"Code": "Code_ets"}
).to_sql(
    "comp_promp_par_ets",
    con=db_ops.civ_engine,
    schema=schema_name,
    index=False,
    if_exists="append",
)

db_ops.civ_engine.dispose()

#### `df_comp_promp_region`

In [50]:
db_ops.civ_cursor.execute(
    f"""
DELETE FROM {schema_name}.comp_promp_attendu_region
WHERE date_report = '{pd.to_datetime(date_report, format="%d-%m-%Y").strftime('%Y-%m-%d')}'
"""
)

db_ops.conn.commit()


df_comp_promp_region = df_comp_promp_region.map(
    lambda x: x.lstrip().rstrip() if isinstance(x, str) else x
)

df_comp_promp_region.merge(
    df_region[["Code_region", "Region"]], on="Region", how="left"
).drop(columns="Region").rename(columns=lambda x: x.replace("région", "region")).to_sql(
    "comp_promp_attendu_region",
    con=db_ops.civ_engine,
    schema=schema_name,
    index=False,
    if_exists="append",
)

db_ops.civ_engine.dispose()

### Table de faits Etat de stocks

#### `recap_stock_by_region`

In [51]:
db_ops.civ_cursor.execute(
    f"""
DELETE FROM {schema_name}.recap_stock_by_region
WHERE date_report = '{pd.to_datetime(date_report, format="%d-%m-%Y").strftime('%Y-%m-%d')}'
"""
)

db_ops.conn.commit()

df_sheet_two.merge(df_region[["Code_region", "Region"]]).drop(columns="Region").to_sql(
    "recap_stock_by_region",
    con=db_ops.civ_engine,
    schema=schema_name,
    index=False,
    if_exists="append",
)

db_ops.civ_engine.dispose()

In [52]:
# Get full data product
full_product = db_ops.get_data_from_database(
    "dim_produit",
)
full_product["Programme"] = full_product["Code_sous_prog"].apply(
    lambda x: x.split("-")[0]
)
full_product["Code_produit"] = full_product["Code_produit"].astype(str)
full_product.head(2)

,id_produit_pk,Code_produit,Produit_designation,Unit_rapportage,Categorie_produit,Categorie_du_produit,Code_sous_prog,Programme
0,1,3010049,PARACETAMOL 100 mg comp. BTE/100 BOITE -,COMPRIME,Produit non traceur,PRODUITS PNLP,PNLP-1,PNLP
1,2,3010062,PARACETAMOL 250 mg comp BTE/100 BOITE -,COMPRIME,Produit non traceur,PRODUITS PNLP,PNLP-1,PNLP


#### `recap_stock_prog_region`

In [53]:
db_ops.civ_cursor.execute(
    f"""
DELETE FROM {schema_name}.recap_stock_prog_region
WHERE date_report = '{pd.to_datetime(date_report, format="%d-%m-%Y").strftime('%Y-%m-%d')}'
"""
)

db_ops.conn.commit()

In [54]:
stock_region["Code"] = stock_region["Code"].astype(str)
stock_region["date_report"] = pd.to_datetime(date_report, format="%d-%m-%Y")
stock_region = stock_region.map(
    lambda x: x.lstrip().rstrip() if isinstance(x, str) else x
)
stock_region["MSD"] = stock_region["MSD"].apply(
    lambda x: str(round(float(x), 1)).replace(".", ",") if x != "NA" else "NA"
)

assert (
    stock_region.merge(
        full_product[["id_produit_pk", "Code_produit", "Programme"]].drop_duplicates(
            subset=["Code_produit", "Programme"]
        ),
        left_on=["Code", "Programme"],
        right_on=["Code_produit", "Programme"],
        how="left",
    )
    .merge(df_region[["Code_region", "Region"]], on="Region", how="left")
    .drop(columns=["Code", "Code_produit", "Region"])
    .shape[0]
    == stock_region.shape[0]
)

In [55]:
stock_region.merge(
    full_product[["id_produit_pk", "Code_produit", "Programme"]].drop_duplicates(
        subset=["Code_produit", "Programme"]
    ),
    left_on=["Code", "Programme"],
    right_on=["Code_produit", "Programme"],
    how="left",
).merge(df_region[["Code_region", "Region"]], on="Region", how="left").drop(
    columns=["Code", "Code_produit", "Region"]
).rename(
    columns={"id_produit_pk": "id_produit_fk"}
).to_sql(
    "recap_stock_prog_region",
    con=db_ops.civ_engine,
    schema=schema_name,
    index=False,
    if_exists="append",
)

db_ops.civ_engine.dispose()

#### `recap_stock_prog_nat`

In [56]:
db_ops.civ_cursor.execute(
    f"""
DELETE FROM {schema_name}.recap_stock_prog_nat
WHERE date_report = '{pd.to_datetime(date_report, format="%d-%m-%Y").strftime('%Y-%m-%d')}'
"""
)

db_ops.conn.commit()

In [57]:
stock_national = stock_lvl_decent[
    [
        "Code",
        "Programme",
        "Region",
        "lvl_decent_msd",
        "lvl_decent_statut",
        "lvl_decent_conso",
        "lvl_decent_sdu",
        "lvl_decent_cmm",
        "dispo_globale",
        "dispo_globale_cible",
        "dispo_traceur",
        "dispo_traceur_cible",
    ]
].rename(
    columns={
        "lvl_decent_msd": "MSD",
        "lvl_decent_statut": "STATUT",
        "lvl_decent_conso": "conso_lvl_national",
        "lvl_decent_sdu": "sdu_lvl_national",
        "lvl_decent_cmm": "cmm_lvl_national",
    }
)

In [58]:
stock_national["Code"] = stock_national["Code"].astype(str)
stock_national = stock_national.map(lambda x: x.strip() if isinstance(x, str) else x)
stock_national["date_report"] = pd.to_datetime(date_report, format="%d-%m-%Y")

stock_national["MSD"] = stock_national["MSD"].apply(
    lambda x: str(round(float(x), 1)).replace(".", ",") if x != "NA" else "NA"
)

In [59]:
assert (
    stock_national.merge(
        full_product[["id_produit_pk", "Code_produit", "Programme"]].drop_duplicates(
            subset=["Code_produit", "Programme"]
        ),
        left_on=["Code", "Programme"],
        right_on=["Code_produit", "Programme"],
        how="left",
    )
    .merge(df_region[["Code_region", "Region"]], on="Region", how="left")
    .drop(columns=["Code", "Code_produit", "Region"])
    .shape[0]
    == stock_national.shape[0]
)

stock_national = (
    stock_national.merge(
        full_product[["id_produit_pk", "Code_produit", "Programme"]].drop_duplicates(
            subset=["Code_produit", "Programme"]
        ),
        left_on=["Code", "Programme"],
        right_on=["Code_produit", "Programme"],
        how="left",
    )
    .merge(df_region[["Code_region", "Region"]], on="Region", how="left")
    .drop(columns=["Code", "Code_produit", "Region"])
    .rename(
        columns={
            "id_produit_pk": "id_produit_fk",
            "conso_lvl_national": "CONSO",
            "sdu_lvl_national": "SDU",
            "cmm_lvl_national": "CMM",
        }
    )
)

df_count_prog = (
    stock_national.groupby(["Programme"])["id_produit_fk"].count().reset_index()
)

stock_national["statut_pourcentage"] = stock_national[["Programme", "STATUT"]].apply(
    lambda row: 1
    / df_count_prog.loc[df_count_prog.Programme == row.Programme, "id_produit_fk"].iloc[
        0
    ],
    axis=1,
)

stock_national.to_sql(
    "recap_stock_prog_nat",
    con=db_ops.civ_engine,
    schema=schema_name,
    index=False,
    if_exists="append",
)

db_ops.civ_engine.dispose()

del df_count_prog

#### `Etat de stock`

In [60]:
db_ops.civ_cursor.execute(
    f"""
DELETE FROM {schema_name}.etat_de_stock
WHERE date_report = '{pd.to_datetime(date_report, format="%d-%m-%Y").strftime('%Y-%m-%d')}'
"""
)

db_ops.conn.commit()

In [61]:
mois_prec = (
    pd.to_datetime(date_report, format="%d-%m-%Y").replace(day=1) - pd.Timedelta(days=1)
).strftime("%Y-%m-%d")

df_mois_prec = pd.read_sql(
    f"select * from {schema_name}.etat_de_stock where date_report='{mois_prec}'",
    db_ops.civ_engine,
)

In [62]:
# Nettoyage initial des données
df_etat_stock["Code_produit"] = df_etat_stock["Code_produit"].astype(str)
df_etat_stock = df_etat_stock.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Suppression des colonnes inutiles et nettoyage supplémentaire
cols_to_drop = ["programme_abrv", "Region", "District", "Structure"]
df_ = df_etat_stock.drop(columns=cols_to_drop)

# Première jointure avec df_sous_prog
df_ = df_.merge(
    df_sous_prog, left_on="sous_programme", right_on="Sous_programme", how="left"
).drop(columns=["Sous_programme", "Programme", "sous_programme"])

# Définition des colonnes pour la déduplication
dedup_cols = [
    "Code_produit",
    "Code_sous_prog",
    "Produit_designation",
    "Unit_rapportage",
    "Categorie_du_produit",
    "Categorie_produit",
]

# Seconde jointure avec full_product
df_ = df_.merge(
    full_product.drop_duplicates(subset=dedup_cols),
    right_on=dedup_cols,
    left_on=[
        "Code_produit",
        "Code_sous_prog",
        "produit_designation",
        "unit_rapportage",
        "cat_du_produit",
        "cat_produit",
    ],
)

# Nettoyage final des colonnes
final_cols_to_drop = [
    "Code_produit",
    "cat_produit",
    "produit_designation",
    "unit_rapportage",
    "cat_du_produit",
    "Code_sous_prog",
    "Produit_designation",
    "Unit_rapportage",
    "Categorie_produit",
    "Categorie_du_produit",
    "Programme",
]

df_ = df_.drop(columns=final_cols_to_drop).rename(
    columns={"id_produit_pk": "id_produit_fk"}
)

In [63]:
df_['Code_ets'] = df_['Code_ets'].astype('Int64')

In [64]:
assert (
    df_.merge(
        df_mois_prec[["Code_ets", "id_produit_fk", "qte_cmde", "etat_stock"]].rename(
            columns={
                "qte_cmde": "qte_cmde_mois_prec",
                "etat_stock": "etat_stock_mois_prec",
            }
        ),
        on=["Code_ets", "id_produit_fk"],
        how="left",
    ).shape[0]
    == df_.shape[0]
)


df_ = df_.merge(
    df_mois_prec[["Code_ets", "id_produit_fk", "qte_cmde", "etat_stock"]].rename(
        columns={"qte_cmde": "qte_cmde_mois_prec", "etat_stock": "etat_stock_mois_prec"}
    ),
    on=["Code_ets", "id_produit_fk"],
    how="left",
)

df_.drop(
    columns=["id_region_esigl", "TYPE DE STRUCTURE", "id_district_esigl"], inplace=True
)

df_.head(3)

,Periode,Code_ets,stock_initial,qte_recue,qte_utilisee,perte_ajust,j_rupture,sdu,cmm_esigl,cmm_gest,qte_prop,qte_cmde,qte_approuv,msd,etat_stock,besoin_cmde_urg,besoin_trsf_in,qte_trsf_out,date_report,id_produit_fk,qte_cmde_mois_prec,etat_stock_mois_prec
0,AVRIL 2025,60100090,10,0,3,-2,0,5,2,3.0,3,7,7,1.666667,ENTRE PCU et MIN,NaN,NaN,NaN,2025-04-30,4982,10.0,BIEN STOCKE
1,AVRIL 2025,60100090,22,0,3,0,0,19,2,2.0,0,0,0,9.500000,SURSTOCK,NaN,NaN,11.0,2025-04-30,5004,0.0,SURSTOCK
2,AVRIL 2025,60100090,0,0,0,0,30,0,0,25.0,0,100,100,0.000000,RUPTURE,100.0,25.0,NaN,2025-04-30,155,NaN,NaN


In [65]:
df_.to_sql(
    "etat_de_stock",
    con=db_ops.civ_engine,
    schema=schema_name,
    index=False,
    if_exists="append",
)

db_ops.civ_engine.dispose()